In [ ]:
import collections 

Union Find + backtrack

In [ ]:
# from collections import List 
from typing import List

class UnionFind:
    # Initialize parents
    def __init__(self):
        self.parent = {}

    # Find root of node x with path compression
    def find(self, word):
        if word not in self.parent:
            self.parent[word] = word
        if self.parent[word] != word:
            self.parent[word] = self.find(self.parent[word])
        return self.parent[word]

    # Merge 2 set in same group
    def union(self, word1, word2):
        root1 = self.find(word1)
        root2 = self.find(word2)

        if root1 != root2:
            self.parent[root2] = root1

    # Check if they are in the same group
    def connected(self, word1, word2):
        return self.find(word1) == self.find(word2)

class Solution:
    def generateSentences(self, synonyms: List[List[str]], text: str) -> List[str]:
        # merge 2 words with root word1 
        union_find = UnionFind()
        for word1, word2 in synonyms:
            union_find.union(word1, word2)


        # build a synonym group by root parents
        synonym_group = {}
        # example parent root: [group]
        # {"happy": ['happy', 'joy', 'cheerful']}
        for word in union_find.parent.keys():
            root = union_find.find(word)
            if root not in synonym_group: synonym_group[root] = []
            synonym_group[root].append(word)

        # Sort synonym group value lists
        for group in synonym_group.values():
            group.sort()


        # To parse text, create another dictionary including value: [group]
        # example: value: [group]
        # cheerful: ['happy', 'joy', 'cheerful']
        # all synonym to extract
        synonyms = {}
        for group in synonym_group.values():
            for word in group:
                if word in group:
                    if word not in synonyms: synonyms[word] = []
                    synonyms[word] = group

        # backtrack to generate sentences
        words = text.split()
        result = []
        current_sentence = []
        def backtrack(index):
            # Goal
            if index == len(words):
                result.append(" ".join(current_sentence))
                return 

            word = words[index]
            # there is synonym in dictionary
            if word in synonyms:
                for synonym in synonyms[word]:
                    current_sentence.append(synonym)
                    backtrack(index+1)
                    current_sentence.pop()
            # there is no synonym. keep it as it is
            else:
                current_sentence.append(word)
                backtrack(index+1)
                current_sentence.pop()
        backtrack(0)

        return sorted(result)

In [ ]:
# Graph BFS - simultaneous level by level
# Rotting Oranges: 
'''
Question: Every minute, any fresh orange that is 4-directionally adjacent to a rotten orange becomes rotten. It means simultanious check by level by level to add to minutes after 1 level before next level
Goal: 
    - Track the time it takes for all fresh oranges to rot, considering level-by-level progression.
    - because all adjacent oranges rot simultaneously
Pattern: is naturally a BFS problem as it requires simultaneous rotting, which BFS handles efficiently.
Why not visited track: 
    - eparate visited set would be redundant since the grid's state already prevents reprocessing of rotted cells.
Why not traverse grid after BFS:
    - After the BFS finishes, there’s no need to traverse the grid again unless you're checking if any fresh oranges (1s) remain.
    - The BFS tracks the rotting process and time in a single pass through the queue.

while queue:
    for _ in range(len(queue)):  # Process all nodes at the current level
        row, col = queue.popleft()
        # Process adjacent oranges and add to the queue
    minutes += 1  # Increment time after processing a level

compare to number of island:
Once BFS completes for one component, we move to the next unvisited 1 in the grid
while queue:
    row, col = queue.popleft()
    # Process adjacent cells and add to the queue
''' 

class Solution:
    '''
    Input, Output
    track of fresh -> no fresh cell == 1
    Level by level: queue length loop
        process adjacent cell
        rot the orange -> grid = 2
    add minutes after each level
    
    Graph BFS - simultaniously
    '''
    def orangesRotting(self, grid: List[List[int]]) -> int:
        # Init
        minute = 0
        # to track
        fresh_counter = 0
        # ROW, COL in grid
        ROW, COL = len(grid), len(grid[0])
        invalid = -1

        # Track of fresh count in grid - traverse the grid
        queue = collections.deque()
        for row in range(ROW):
            for col in range(COL):
                # check fresh 
                if grid[row][col] == 1:
                    fresh_counter += 1
                # add initial rotten orange
                elif grid[row][col] == 2:
                    queue.append((row, col))

        # edge case - no fresh orange
        if not fresh_counter: return 0

        # BFS - level by level
        direction = [(0,1),(0,-1),(1,0),(-1,0)]
        while queue:
            n = len(queue)
            # check each level
            print(queue)
            for _ in range(n):
                # remove first cell
                cur_row, cur_col = queue.popleft()
                # process directions in each cell
                for dir_row, dir_col in direction:
                    new_row, new_col = cur_row + dir_row, cur_col + dir_col

                    # check limits
                    if (new_row >= ROW or new_row < 0
                        or new_col >= COL or new_col <0
                        or grid[new_row][new_col] != 1):
                        # skip
                        continue

                    # if orange is fresh, mark as rotten
                    grid[new_row][new_col] = 2
                    queue.append((new_row, new_col))
                    # decrement fresh counter
                    fresh_counter -= 1

            # track time
            if queue:
                minute += 1

        return minute if fresh_counter == 0 else invalid
    
'''
# visualization

grid = [
    [2, 1, 1],
    [1, 1, 0],
    [0, 1, 1]
]

Initial
Minute = 0
Queue: [(0, 0)]  # Start with the rotten orange at (0, 0)
Fresh Count: 5

Min 1:
[
    [2, 2, 1],
    [2, 1, 0],
    [0, 1, 1]
]
Queue: [(0, 1), (1, 0)]
Fresh Count: 3

Min 2: 
[
    [2, 2, 2],
    [2, 2, 0],
    [0, 1, 1]
]
Queue: [(0, 2), (1, 1)]

Min 3:
[
    [2, 2, 2],
    [2, 2, 0],
    [0, 2, 1]
]
Queue: [(2, 1)]

Min 4:
[
    [2, 2, 2],
    [2, 2, 0],
    [0, 2, 2]
]
Queue: []

Output: 4
'''

Find Overlaps

In [ ]:
import heapq

class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:
        # find overlaps
        # record all overlaps in heap

        # edge case
        if not intervals: return 0

        # Sort intervals based on start time
        intervals.sort(key=lambda x:x[0])

        # Min heap for end time to track
        min_heap = []
        heapq.heappush(min_heap, intervals[0][1])

        # scan intervals to find overlaps -> equal to room
        for i in range(1, len(intervals)):
            # remove non overlap intervals = if current start is after end of earliest min heap
            if intervals[i][0] >= min_heap[0]:
                heapq.heappop(min_heap)

            # add current meeting end time to min heap
            heapq.heappush(min_heap, intervals[i][1])
        
        return len(min_heap)
    
'''
sorted array for start time
min heap for end time 

For min heap in each loop
before:  [30] 5
after pop:  [30]
after push:  [10, 30]
before:  [10, 30] 15
after pop:  [30]
after push:  [20, 30]

Time: O(nlogk + nlogn) heap operation + sort
- k heap size 

Space: O(k) 
'''

# another aproach: 2 pointer
class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:
        # two pointers
        '''
        Steps:
        Extract start and end times:
        start_meeting = [0, 5, 15]
        end_meeting = [10, 20, 30]

        Traverse:
        Start at 0, end at 10: Add a room → room = 1.
        Start at 5, end at 10: Add a room → room = 2.
        Start at 15, end at 10: Free a room → room = 1.
        Start at 15, end at 20: Add a room → room = 2.
        All meetings processed.

        Max rooms used = 2.

        Visualize:
        start: 0, 5, 15
        end:   10, 20, 30
        0|----------------------|30
            5|--|10  15|--|20

        Time: O(nlogn) sorted
        Space: O(n)
        '''
        # edge case
        if not intervals: return 0

        # initial
        room = 0
        max_room = room
        # start pointer
        start = 0
        # end poiner
        end = 0

        # sorted intervals
        start_meeting = sorted(start_time for start_time, _ in intervals)
        end_meeting = sorted(end_time for _, end_time in intervals)

        # traverse start and end
        while start < len(start_meeting):
            # start meeting
            if start_meeting[start] < end_meeting[end]:
                # add room count
                room += 1
                # shift
                start += 1
                max_room = max(max_room, room)
            # end meeting
            else:
                # room is available
                room -= 1
                end += 1

        return max_room


Top K Frequent Element

In [ ]:
# Heap - Priority Queue
from collections import Counter
from heapq import heappush, heappop
from typing import List

class Solution:
    '''
    Heap:
    - By removing the smallest frequency element, we ensure the heap always contains the k most frequent elements.
    - heapq does not guarantee a sorted order in the entire structure
    - while heapq ensures the root is the smallest element, the rest of the elements may not be in strict sorted order. (parent ≤ children)

    Example:
    [1,1,1,2,2,3] -> 1:3, 2:2, 3:1 -> heap [(1,3),(3,1),(2,2)] -> k = 2 -> remove min root -> [(2, 2), (3,1)]

    Time: O(n + m.logk) 
    - n Counter
    - m unique element in dictionary
    - log k heappush and heappop operations in heap
    Space: O(m + k)
    - m counter 
    - k heap  
    '''
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        # Count frequency - dic
        count_frequency = Counter(nums)

        # min heap
        min_heap = []
        # iterate
        for num, count in count_frequency.items():
            # add num to heap
            heappush(min_heap, (count, num))
            # pop if it pass the window length k
            if k < len(min_heap):
                # remove min from priority queue
                heappop(min_heap)

        # if we need to extract top k element
        return [num for _, num in min_heap]
    
# Bucket sort
'''
nums = [1, 1, 1, 2, 2, 3]
k = 2

count_frequency = {1: 3, 2: 2, 3: 1}

buckets = [
    [],          # Frequency 0 (unused)
    [3],         # Frequency 1
    [2],         # Frequency 2
    [1]          # Frequency 3
]

frequency * unique num = n length of nums
'''
class Solution:
    '''
    bucket sort:
    frequency is index -> less to high frequency based on 0 index to ..
    - Create a list of buckets where the index represents the frequency. Each bucket stores the numbers that appear with that frequency.
    - Iterate over the count_frequency dictionary and place each number into the appropriate bucket.

    Time: O(n)
    total number of iterations is proportional to n, the size of the input
    n freq -> [index 0: [], index 1 freq: [nums[0] to n]] -> O(n)
    1 freq -> [0-> [], 1 -> [1 element], 2 -> [1 element], to n element]
    Space: O(m + n)
    - count frequency dictionary - unique num m
    - bucket array: index: frequency and num in each index -> frequency size n
    '''
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        # bucket sort
        # count frequency
        count_frequency = Counter(nums)

        # max range of frequency
        max_frequency = len(nums)

        # construct bucket array that index is frequency
        bucket = [[] for _ in range(max_frequency + 1)]
        for num, count in count_frequency.items():
            bucket[count].append(num)


        result = []
        # extract k high frequency by high priority -> reverse
        for frequency in range(max_frequency + 1 - 1,  0, -1):
            for num in bucket[frequency]:
                result.append(num)
                if len(result) == k:
                    return result